This notebook refers heavily to these resources: 

1. https://www.youtube.com/watch?v=UU1WVnMk4E8&t=1146s&ab_channel=freeCodeCamp.orgOpen 

2. https://github.com/Infatoshi/fcc-intro-to-llms 

We are going to use torch library. This allows us to use tensor operation

# Prepare the training data
The training text file

In [10]:
import torch
import torch.nn as nn
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Running torch on {}'.format(device))
print(torch.version.cuda)
with open('wizard_of_oz.txt', 'r', encoding='utf-8') as f:
    text = f.read()
print(text[:200])

Running torch on cuda
12.1
﻿The Project Gutenberg eBook of The Wonderful Wizard of Oz
    
This ebook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restri


# 1. Tokenizer

A tokenizer coverts a text into a mathematical object that is easier for the computer to process. There are 3 different level of tokenizers:

1. Word tokenizer: This type of tokenization splits text into individual words. For example, the sentence "Hello, world!" would be tokenized into ["Hello", ",", "world", "!"]. Word tokenization is commonly used in many NLP tasks.

2. Subword Tokenizer: Subword tokenization breaks down words into smaller units called subwords. This approach is particularly useful for handling languages with complex morphology or dealing with out-of-vocabulary words. Examples of subword tokenization algorithms include Byte Pair Encoding (BPE) and WordPiece.

3. Character Tokenizer: Character tokenization represents each character in the text as a separate token. This approach is useful when dealing with languages that do not have clear word boundaries or for tasks like text generation.


Here, we will use the simplest tokenizer, the character tokenizer. We can build our own tokenizer through the code written below:

In [2]:
chars = sorted(set(text))
print(chars)
vocubary_size = len(chars)
string_to_int = { ch:i for i, ch in enumerate(chars)}
int_to_string = { i:ch for i, ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: "".join([int_to_string[i] for i in l ])

print('Encoding ...: {}'.format(encode('hello')))
print('Decoding ... {}'.format(decode(encode('hello'))))


['\n', ' ', '!', '#', '$', '%', '&', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']
Encoding ...: [62, 59, 66, 66, 69]
Decoding ... hello


Once the encoding function is defined, we shall encode our text. Since the text contains a lot of characters, after the encoding our list will be super large, which will not be efficient to operate on. Instead, we are going to store all the encoded text data into a torch.tensor data structure. Storing the data into tensor will allow us to perform tensor operation on it. 

In [3]:
data = torch.tensor(encode(text), dtype = torch.long)
print(data[:100])

tensor([88, 46, 62, 59,  1, 42, 72, 69, 64, 59, 57, 74,  1, 33, 75, 74, 59, 68,
        56, 59, 72, 61,  1, 59, 28, 69, 69, 65,  1, 69, 60,  1, 46, 62, 59,  1,
        49, 69, 68, 58, 59, 72, 60, 75, 66,  1, 49, 63, 80, 55, 72, 58,  1, 69,
        60,  1, 41, 80,  0,  1,  1,  1,  1,  0, 46, 62, 63, 73,  1, 59, 56, 69,
        69, 65,  1, 63, 73,  1, 60, 69, 72,  1, 74, 62, 59,  1, 75, 73, 59,  1,
        69, 60,  1, 55, 68, 79, 69, 68, 59,  1])


# 2. Preparing the data for the network training

We are going to split the text file into training data and validation data. This is normally done through a 8:2 split. The splitting is important to make the model generative instead of spliting out the whole text that it has been fed. 

Feeding the whole chunck of text data to the computer and letting it iterate one by one can be very time consuming. To save time, we can divide the training data into batches and let the computer processes these batches of data at the same time. This is known as parallelism, and will help us to save up a lot of computational time. 

In language model, a block size refers to the length of the sequences or blocks of text used during training. Sequences longer than the block size will be divided into smaller blocks or truncated. Therefore, block size determines the maximum length of sequences that the model can handle during training. 

We will define a function that randomly select a block of text from the training/validation data. The output of this function will consist of the random block of text selected from the data stored, as well as a block of text that follows immediately from the selected data, both output stored as torch.tensor. 

In [9]:
block_size = 8
batch_size = 4

n = int(0.8*len(data)) # spliting the data 
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    '''
    Randomly obtain a chunck of input data from either the train set or validation set. 
    Store the next sequence to a tensor (y).
    Output the input and output data 
    '''
    data = train_data if split == 'train' else val_data
    print(len(data) - block_size)
    ix = torch.randint(len(data) - block_size, (batch_size,)) 
    x = torch.stack([data[i:i+block_size] for i in ix]) 
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

x, y = get_batch('train')
print('inputs:')
# print(x.shape)
print(x)
print('targets:')
print(y)


181672
inputs:
tensor([[59,  1, 58, 69, 69, 72,  1, 69],
        [72,  1, 58, 69, 77, 68,  1, 75],
        [69, 55, 74, 73,  1, 77, 59, 72],
        [72, 68, 59, 73, 74, 66, 79, 12]], device='cuda:0')
targets:
tensor([[ 1, 58, 69, 69, 72,  1, 69, 60],
        [ 1, 58, 69, 77, 68,  1, 75, 70],
        [55, 74, 73,  1, 77, 59, 72, 59],
        [68, 59, 73, 74, 66, 79, 12,  1]], device='cuda:0')


# Writing the model

Now that the training data is in place, we are going to write the training model. 

1. Embedding layer

The first thing in the forward function is to perform embedding. An embedding is a vector representation of a discrete categorical variable, typically used to represent words or entities in natural language processing (NLP) tasks. It maps each categorical value to a high-dimensional vector in a continuous space, where similar values are represented by similar vectors.

In the context of natural language processing, embedding refers to the action of represent words as dense vectors in a continuous vector space, where the position of each word vector is learned based on its contextual usage in a large corpus of text. Word embeddings capture semantic relationships between words, allowing similar words to have similar vector representations. Through embedding, we can store more information aside from the character itself (for example, its positional relation to others). In this model, the embedding is being carried out by the torch module torch.nn.Embedding(vocab_size, vocab_size)

3. Activation function 

An activation function assigns weight to each input data after passing through a layer. Common example includes a linear activation, ReLU activation etc. 
 

In [ ]:
class languagemodel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)
    
    def forward(self, index, targets=None):
        logits = self.token_embedding_table(index) # perform embedding to the word and store it as logits
        
        return logits
    